In [3]:
import numpy as np
import pyphi
# from pip install "git+https://github.com/wmayner/pyphi@feature/iit-4.0#egg=pyphi"
from qutip import * #4.7.1
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
pyphi.config.PROGRESS_BARS = False

pyphi.config.PARALLEL_COMPLEX_EVALUATION = False
pyphi.config.PARALLEL_CONCEPT_EVALUATION = False
pyphi.config.PARALLEL_CONCEPT_EVALUATION = False
pyphi.config.PARALLEL_CUT_EVALUATION = False
pyphi.config.PARALLEL_PURVIEW_EVALUATION = 4.0
pyphi.config.PARALLEL_MECHANISM_PARTITION_EVALUATION = False

# Define states and matrices

In [6]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()
superposition = (1/3*zero + 2/3*one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_sp = superposition * superposition.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

rho_mix = 0.5*(tensor(rho_zero, rho_zero) + tensor(rho_one, rho_one))
rho_bell = ket2dm(bell_state(state = '00'))
rho_GHZ = ket2dm(ghz_state(N=3))
rho_W = ket2dm(w_state(N=3))
rho_bellmix = 1/3*(tensor(rho_bell, rho_zero) + tensor(rho_zero, rho_bell) + tensor(rho_zero, rho_bell).permute([1,0,2]))

rho_bennet = 1/(3-4)*(1-(tensor(rho_zero, rho_one, rho_plus) + tensor(rho_one, rho_plus, rho_zero) 
                        + tensor(rho_plus, rho_zero, rho_one) + tensor(rho_minus, rho_minus, rho_minus)))


In [7]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([1.+0.j, 0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# Classical results

In [8]:
qgate = cnot()
state = (1,0)
node_labels = ("A", "B")
Qtpm = qgate.data.toarray()

In [9]:
Qtpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [10]:
# pyphi convention for state ordering is little endian
Qtpm = pyphi.convert.be2le_state_by_state(Qtpm.real)

In [11]:
network = pyphi.Network(Qtpm, node_labels=node_labels)
subsystem = pyphi.Subsystem(network, state)

In [12]:
network.tpm.print()

(0, 0): [0. 0.]
(1, 0): [1. 1.]
(0, 1): [0. 1.]
(1, 1): [1. 0.]


In [13]:
ces = pyphi.new_big_phi.phi_structure(subsystem, parallel = False)
ces.distinctions

══════════════════════════════════════════════════════════════════════════════════
                     Cause-effect structure (2 distinctions)                      
══════════════════════════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
                    Distinction: mechanism = [A], state = [1]                     
                                      φ = 1                                       
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
  ┌─────────────────────────────────────┐┌─────────────────────────────────────┐  
  │  Maximally-irreducible cause        ││  Maximally-irreducible effect       │  
  │    φ = 1                            ││    φ = 1                            │  
  │    Normalized φ = 1                 ││    Normalized φ = 1                 │  
  │    Mechanism: [A]                   ││    Mechanism: [A]                   │  
  │ 

In [14]:
# Compute effect of mechanism (A, B) and show ties in purview --> we select the largest purview
subsystem.mie((0, 1)).purview_ties

(Maximally-irreducible effect
   φ = 1
   Normalized φ = 1/2
   Mechanism: [A,B]
   Purview: [B]
   Specified state:
   ┌──────────────────┐
   │ Specified EFFECT │
   │  ╍╍╍╍╍╍╍╍╍╍╍╍╍╍  │
   │  EFFECT:  (1,)   │
   │    II_e: 1.0     │
   └──────────────────┘
   Direction: EFFECT
   MIP:
     A,B    ∅ 
     ─── ✕ ───
      ∅     B 
   Forward probability: 
       1.0
   Partitioned forward probability:
       0.5
   #(state ties): 0
   #(partition ties): 1
   #(purview ties): 1,
 Maximally-irreducible effect
   φ = 1
   Normalized φ = 1/3
   Mechanism: [A,B]
   Purview: [A,B]
   Specified state:
   ┌──────────────────┐
   │ Specified EFFECT │
   │ ╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍ │
   │ EFFECT:  (1, 1)  │
   │   II_e: 2.0      │
   └──────────────────┘
   Direction: EFFECT
   MIP:
      A     B     ∅ 
     ─── ✕ ─── ✕ ───
      A     ∅     B 
   Forward probability: 
       1.0
   Partitioned forward probability:
       0.5
   #(state ties): 0
   #(partition ties): 0
   #(purview ties): 1)

In [15]:
# find cause of mechanism B
# since mechanism B has a cause but no effect is was not included in the cause-effect structure above
subsystem.find_mice(pyphi.direction.Direction.CAUSE, (1,))

Maximally-irreducible cause
  φ = 1/2
  Normalized φ = 1/4
  Mechanism: [B]
  Purview: [A,B]
  Specified state:
  ┌─────────────────┐
  │ Specified CAUSE │
  │ ╍╍╍╍╍╍╍╍╍╍╍╍╍╍╍ │
  │ CAUSE:  (0, 0)  │
  │  II_c: 0.5      │
  └─────────────────┘
  Direction: CAUSE
  MIP:
     B     ∅ 
    ─── ✕ ───
     ∅    A,B
  Forward probability: 
      1.0
  Partitioned forward probability:
      0.5
  #(state ties): 1
  #(partition ties): 0
  #(purview ties): 0

# Quantum results

## 2 Qubit

In [15]:
qgate = cnot()

In [16]:
# Figure 3
rho = tensor(rho_one, rho_zero)
# rho = tensor(rho_minus, rho_plus)
# rho = tensor(rho_plus, rho_zero)
# rho = tensor(rho_zero, rho_plus)

# Figure 4
# rho = rho_mix

In [17]:
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]

In [18]:
rho_e = evolve_unitary(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]

In [19]:
rho_e.ptrace([1,])

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

In [20]:
result_effect = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  1.0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  1.0
p:  (0, 1)  phi:  1.0


In [21]:
print_QIIT_results(result_effect)

mechanism:  (0,) ; purview:  (0,) ; phi =  1.0
mip: 
 0     ∅ 
─── ✕ ───
 ∅     0 
[Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]]

mechanism:  (0, 1) ; purview:  (0, 1) ; phi =  1.0
mip: 
 0     1     ∅ 
─── ✕ ─── ✕ ───
 0     ∅     1 
[Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]]]



In [22]:
result_cause = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  1.0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  1.0


In [23]:
print_QIIT_results(result_cause)

mechanism:  (0,) ; purview:  (0,) ; phi =  1.0
mip: 
 0     ∅ 
─── ✕ ───
 ∅     0 
[Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]]

mechanism:  (1,) ; purview:  (0, 1) ; phi =  0.5
mip: 
 1     ∅ 
─── ✕ ───
 ∅    0,1
[Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]], Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]]

mechanism:  (0, 1) ; purview:  (0, 1) ; phi =  1.0
mip: 
 0     1     ∅ 
─── ✕ ─── ✕ ───
 0     ∅     1 
[Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]]



## 3 Qubits

In [24]:
# Figure 4
qgate = tensor(qeye(2), cnot())

In [25]:
rho = rho_GHZ
rho

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.  0.  0.  0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.5]]

In [26]:
rho_e = evolve_unitary(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]

In [27]:
rho_e.ptrace([0,1])

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]

In [28]:
entanglement_partition(rho)

[(0, 1, 2)]

In [29]:
entanglement_partition(rho_e)

[(2,), (0, 1)]

In [30]:
result_effect = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  1.0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0.5
p:  (1, 2)  phi:  0.5
p:  (0, 1, 2)  phi:  0.25
m:  (0, 1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  2.0
p:  (0, 2)  phi:  0

In [31]:
print_QIIT_results(result_effect)

mechanism:  (1, 2) ; purview:  (2,) ; phi =  1.0
mip: 
1,2    ∅ 
─── ✕ ───
 ∅     2 
[Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]]

mechanism:  (0, 1, 2) ; purview:  (0, 1, 2) ; phi =  2.0
mip: 
 0    1,2    ∅ 
─── ✕ ─── ✕ ───
 ∅     2    0,1
[Quantum object: dims = [[8], [1]], shape = (8, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]]]



In [32]:
result_cause = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.5
p:  (0, 1, 2)  phi:  0.25
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.5
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  1.0
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p

In [34]:
print_QIIT_results(result_cause)

mechanism:  (2,) ; purview:  (1, 2) ; phi =  0.5
mip: 
 2     ∅ 
─── ✕ ───
 ∅    1,2
[Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]]]

mechanism:  (0, 1) ; purview:  (0, 1, 2) ; phi =  1.0
mip: 
0,1     ∅  
─── ✕ ─────
 ∅    0,1,2
[Quantum object: dims = [[8], [1]], shape = (8, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]], Quantum object: dims = [[8], [1]], shape = (8, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]]]

mechanism:  (0, 1, 2) ; purview:  (0, 1, 2) ; phi =  2.0
mip: 
0,1    2     ∅ 
─── ✕ ─── ✕ ───
 ∅    1,2    0 
[Quantum object: dims = [[8], [1]], shape = (8, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.       

## 3 Qubit Identity

In [35]:
# Figure 5
qgate = tensor(qeye(2),qeye(2),qeye(2))

In [36]:
# rho = tensor(rho_zero, rho_zero, rho_zero)
# rho = rho_GHZ
rho = rho_W
rho

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]

In [37]:
result_effect = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0.276692
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.138346
p:  (0, 2)  phi:  0.138346
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.069173
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.276692
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.138346
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.138346
p:  (0, 1, 2)  phi:  0.069173
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0.276692
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0.138346
p:  (1, 2)  phi:  0.138346
p:  (0, 1, 2)  phi:  0.069173
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.943358
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.471679
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0.943358
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.471679
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.943358
p:  (0, 1, 2)  phi:  0

In [38]:
print_QIIT_results(result_effect)

mechanism:  (0,) ; purview:  (0,) ; phi =  0.276692
mip: 
 0     ∅ 
─── ✕ ───
 ∅     0 
[Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]]

mechanism:  (1,) ; purview:  (1,) ; phi =  0.276692
mip: 
 1     ∅ 
─── ✕ ───
 ∅     1 
[Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]]

mechanism:  (2,) ; purview:  (2,) ; phi =  0.276692
mip: 
 2     ∅ 
─── ✕ ───
 ∅     2 
[Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]]

mechanism:  (0, 1) ; purview:  (0, 1) ; phi =  0.943358
mip: 
0,1    ∅ 
─── ✕ ───
 ∅    0,1
[Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]]

mechanism:  (0, 2) ; purview:  (0, 2) ; phi =  0.943358
mip: 
0,2    ∅ 
─── ✕ ───
 ∅    0,2
[Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]]

mechanism:  (1